In [1]:
import numpy as np

def generate_sequences(n=128, variable_len=False, seed=13):
    basic_corners = np.array([[-1, -1], [-1, 1], [1, 1], [1, -1]])
    np.random.seed(seed)
    bases = np.random.randint(4, size=n)
    if variable_len:
        lengths = np.random.randint(3, size=n) + 2
    else:
        lengths = [4] * n
    directions = np.random.randint(2, size=n)
    points = [basic_corners[[(b + i) % 4 for i in range(4)]][slice(None, None, d*2-1)][:l] + np.random.randn(l, 2) * 0.1 for b, d, l in zip(bases, directions, lengths)]
    return points, directions

In [2]:
import torch
from torch import optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

In [3]:
points, directions = generate_sequences()

In [10]:
X = torch.from_numpy(np.array(points)).float()
y = torch.from_numpy(directions).float()

In [11]:
data = TensorDataset(X, y)

In [16]:
data_loader = DataLoader(data, batch_size=3, shuffle=True)

In [19]:
X_train, y_train = next(iter(data_loader))

In [20]:
X_train.shape, y_train.shape

(torch.Size([3, 4, 2]), torch.Size([3]))

In [39]:
rnn_layer = nn.RNN(input_size=2, hidden_size=2)

In [40]:
out, context_vector = rnn_layer(X_train.permute(1, 0, 2))

In [41]:
out[-1]

tensor([[ 0.3067, -0.6698],
        [-0.8146,  0.1910],
        [-0.7904,  0.6421]], grad_fn=<SelectBackward0>)

In [42]:
context_vector

tensor([[[ 0.3067, -0.6698],
         [-0.8146,  0.1910],
         [-0.7904,  0.6421]]], grad_fn=<StackBackward0>)

In [43]:
rnn_layer = nn.RNN(input_size=2, hidden_size=2, batch_first=True)

In [44]:
out, context_vector = rnn_layer(X_train)

In [49]:
out.permute(1, 0, 2)[-1]

tensor([[-0.3942,  0.9124],
        [-0.2779,  0.5343],
        [-0.0702,  0.5199]], grad_fn=<SelectBackward0>)

In [47]:
context_vector

tensor([[[-0.3942,  0.9124],
         [-0.2779,  0.5343],
         [-0.0702,  0.5199]]], grad_fn=<StackBackward0>)

In [51]:
rnn_layer = nn.RNN(input_size=2, hidden_size=2, batch_first=True, bidirectional=True)

In [55]:
rnn_layer(X_train)[0].shape

torch.Size([3, 4, 4])

In [56]:
rnn_layer(X_train)[1].shape

torch.Size([2, 3, 2])